In [4]:
import pandapower as pp
import pandapower.networks
import pandapower.contingency
import time, json, os
from concurrent.futures import ThreadPoolExecutor

# TODO: 需要本地做映射库, 映射模型ID与真实计算ID
# TODO: 所有必传参数都不能为空值
# TODO: 开关未能正确加载
# TODO: 电网可以本地生成sqlite库, 调整后再加载

bus_map = {}
line_map = {}
trafo_map = {}


def load_file(file_name):
    file_path = f"./断面数据/{file_name}.json"
    if os.path.exists(file_path):
        with open(file_path, "r", encoding="utf-8") as file:
            return json.load(file)
    else:
        return []


def load_element_data(api_endpoint):
    if api_endpoint == "api/bus":
        return load_file("母线")
    if api_endpoint == "api/绕组":
        return load_file("绕组")
    if api_endpoint == "api/线端":
        return load_file("线端")
    elif api_endpoint == "api/trafo_2":
        return load_file("变压器-双")
    elif api_endpoint == "api/trafo_3":
        return load_file("变压器-三")
    elif api_endpoint == "api/line":
        return load_file("交流线路")
    elif api_endpoint == "api/gen":
        return load_file("机组")
    elif api_endpoint == "api/static-gen":
        return load_file("静态机组")
    elif api_endpoint == "api/load":
        return load_file("负荷")
    elif api_endpoint == "api/switch":
        return load_file("开关")
    elif api_endpoint == "api/switch_i":
        return load_file("母线-线路开关")
    elif api_endpoint == "api/switch_t":
        return load_file("母线-变压器开关")
    elif api_endpoint == "api/switch_b":
        return load_file("母线-母线开关") + load_file("母线-母线开关 500+")
    else:
        return []


def load_measurement_data(api_endpoint):
    return []


def load_bus_section(net):
    print("Starting Bus Section")
    bus_data = load_element_data("api/bus")
    for bus in bus_data:
        index = pp.create_bus(
            net,
            name=bus["NAME"],
            vn_kv=float(bus["VN_KV"]),
            max_vm_pu=float(bus["MAX_VM_PU"]),
            min_vm_pu=float(bus["MIN_VM_PU"]),
        )
        bus_map[int(bus["INDEX"])] = int(index)
    for bus in load_element_data("api/绕组"):
        index = pp.create_bus(
            net,
            name=bus["NAME"],
            vn_kv=float(bus["电压等级"]),
            max_vm_pu=None,
            min_vm_pu=None,
        )
        bus_map[int(bus["ID"])] = index
    for bus in load_element_data("api/线端"):
        index = pp.create_bus(
            net,
            name=bus["NAME"],
            vn_kv=float(bus["电压等级"]),
            max_vm_pu=None,
            min_vm_pu=None,
        )
        bus_map[int(bus["ID"])] = index

    # 加载量测数据
    bus_measurements = load_measurement_data("api/measurement/bus")
    print("Bus measurements loaded.")
    print("Bus Section completed\n")


def load_trafo_section(net):
    print("Starting Transformer Section")
    trafo2_data = load_element_data("api/trafo_2")
    for tranfo in trafo2_data:
        index = pp.create_transformer_from_parameters(
            net,
            # index=int(tranfo["ID"]),
            name=tranfo["NAME"],
            hv_bus=bus_map[int(tranfo["HV_BUS"])],
            lv_bus=bus_map[int(tranfo["LV_BUS"])],
            sn_mva=999,
            vn_hv_kv=999,
            vn_lv_kv=999,
            vkr_percent=999,
            vk_percent=999,
            pfe_kw=999,
            i0_percent=999,
            max_loading_percent=110
        )
        trafo_map[int(tranfo["ID"])] = int(index)
    trafo3_data = load_element_data("api/trafo_3")
    for tranfo in trafo3_data:
        index = pp.create_transformer3w_from_parameters(
            net,
            # index=int(tranfo["ID"]),
            name=tranfo["NAME"],
            hv_bus=bus_map[int(tranfo["HV_BUS"])],
            mv_bus=bus_map[int(tranfo["MV_BUS"])],
            lv_bus=bus_map[int(tranfo["LV_BUS"])],
            sn_mva=int(tranfo["SN_MVA"]),
            sn_hv_mva=int(tranfo["SN_MVA"]),
            sn_mv_mva=int(tranfo["SN_MVA"]),
            sn_lv_mva=int(tranfo["SN_MVA"]),
            vn_hv_kv=int(tranfo["VN_HV_KV"]),
            vn_mv_kv=int(tranfo["VN_MV_KV"]),
            vn_lv_kv=int(tranfo["VN_LV_KV"]),
            vkr_percent=float(tranfo["VKR_PERCENT"]),
            vkr_hv_percent=float(tranfo["VKR_PERCENT"]),
            vkr_mv_percent=float(tranfo["VKR_PERCENT"]),
            vkr_lv_percent=float(tranfo["VKR_PERCENT"]),
            vk_percent=float(tranfo["VK_PERCENT"]),
            vk_hv_percent=float(tranfo["VK_PERCENT"]),
            vk_mv_percent=float(tranfo["VK_PERCENT"]),
            vk_lv_percent=float(tranfo["VK_PERCENT"]),
            pfe_kw=int(tranfo["PFE_KW"]),
            i0_percent=float(tranfo["I0_PERCENT"]),
            max_loading_percent=110
        )
        trafo_map[int(tranfo["ID"])] = int(index)
    # 加载量测数据
    trafo_measurements = load_measurement_data("api/measurement/trafo")
    print("Transformer measurements loaded.")
    print("Transformer Section completed\n")


def load_line_section(net):
    print("Starting Line Section")
    line_data = load_element_data("api/line")
    for line in line_data:
        try:
            index = pp.create_line_from_parameters(
                net,
                # index=int(line["ID"]),
                name=line["NAME"],
                from_bus=bus_map[int(line["FROM_BUS"])],
                to_bus=bus_map[int(line["TO_BUS"])],
                length_km=float(line["LENGTH_KM"]),
                r_ohm_per_km=float(line["R_OHM_PER_KM"]),
                x_ohm_per_km=float(line["X_OHM_PER_KM"]),
                c_nf_per_km=float(line["C_NF_PER_KM"]),
                max_i_ka=float(line["MAX_I_KA"]),
                max_loading_percent=120
            )
            line_map[int(line["ID"])] = int(index)
        except Exception as e:
            print(line, e)
    # 加载量测数据
    line_measurements = load_measurement_data("api/measurement/line")
    print("Line measurements loaded.")
    print("Line Section completed\n")


def load_static_gen_section(net):
    print("Starting Static Generator Section")
    gen_data = load_element_data("api/static-gen")
    # print('static gen-data', gen_data)
    for gen in gen_data:
        try:
            print(gen["BUS"], gen["NAME"], "static-gen-bus")
            pp.create_sgen(
                net,
                index=int(gen["ID"]),
                name=gen["NAME"],
                bus=bus_map[int(gen["BUS"])],
                p_mw=float(gen["P_MW"]),
                q_mvar=float(gen["Q_MVAR"]),
            )
        except Exception as e:
            print(gen, e)
    # 加载量测数据
    gen_measurements = load_measurement_data("api/measurement/static-gen")
    print("Generator measurements loaded.")
    print("Generator Section completed\n")


def load_gen_section(net):
    print("Starting Generator Section")
    gen_data = load_element_data("api/gen")
    for gen in gen_data:
        try:
            pp.create_gen(
                net,
                index=int(gen["ID"]),
                name=gen["NAME"],
                bus=int(gen["母线ID"]),
                p_mw=float(gen["标称功率"]),
            )
        except Exception as e:
            print(gen, e)
    # 加载量测数据
    gen_measurements = load_measurement_data("api/measurement/gen")
    print("Generator measurements loaded.")
    print("Generator Section completed\n")


def load_load_section(net):
    print("Starting Load Section")
    load_data = load_element_data("api/load")
    for load in load_data:
        pp.create_load(
            net,
            index=load["ID"],
            name=load["NAME"],
            bus=bus_map[int(load["BUS"])],
            p_mw=float(load["P_MW"]),
            q_mvar=float(load["Q_MVAR"]),
        )
    # 加载量测数据
    load_measurements = load_measurement_data("api/measurement/load")
    print("Load measurements loaded.")
    print("Load Section completed\n")


def load_switch_section(net):
    print("Starting Switch Section")
    for switch in load_element_data("api/switch"):
        if switch["ET"] == "b":
            element = bus_map[int(switch["ELEMENT"])]
        elif switch["ET"] == "l":
            element = line_map[int(switch["ELEMENT"])]
        elif switch["ET"] == "t" or switch['ET'] == 't3':
            element = trafo_map[int(switch["ELEMENT"])]
        pp.create_switch(
            net,
            # index=switch["ID"],
            name=switch["NAME"],
            bus=bus_map[int(switch["BUS"])],
            et=switch["ET"],
            element=element,
        )
    # 加载量测数据
    switch_measurements = load_measurement_data("api/measurement/switch")
    print("Switch measurements loaded.")
    print("Switch Section completed\n")


def check_network_data(net):
    print("Starting network data check")

    # 检查并删除无效引用
    elements_with_bus = [
        "line",
        "trafo",
        "switch",
        "load",
        "gen",
        "sgen",
        "ward",
        "xward",
    ]

    for element in elements_with_bus:
        if element in net and not net[element].empty:
            if element == "switch":
                # 开关特殊处理：使用bus和element字段
                bus_columns = ["bus"]
                # 检查element字段是否引用了有效的元素（根据et类型）
                # 这部分需要更复杂的逻辑，暂时只检查bus字段
            elif element in ["line", "trafo"]:
                from_bus_col = "hv_bus" if element == "trafo" else "from_bus"
                to_bus_col = "lv_bus" if element == "trafo" else "to_bus"
                bus_columns = [from_bus_col, to_bus_col]
            else:
                bus_columns = ["bus"]

            for bus_col in bus_columns:
                invalid_ids = net[element][
                    ~net[element][bus_col].isin(net.bus.index)
                ].index
                if len(invalid_ids) > 0:
                    print(
                        f"Deleting {len(invalid_ids)} invalid entries in {element} (invalid {bus_col})"
                    )
                    net[element].drop(invalid_ids, inplace=True)

    # 先重置母线索引
    pp.create_continuous_bus_index(net)
    # 重置所有元件索引, 避免内存溢出
    pp.toolbox.create_continuous_elements_index(net)
    pp.runpp(net)
    pp.to_sqlite(net, "./net.db")
    print("network data check completed and power flow passed")

In [5]:

# 创建空网络
net = pp.create_empty_network(name="浙江电网")

# 环节1：加载母线
load_bus_section(net)

# 环节2-5：并行执行
with ThreadPoolExecutor() as executor:
    futures = [
        executor.submit(load_trafo_section, net),
        executor.submit(load_line_section, net),
        executor.submit(load_gen_section, net),
        executor.submit(load_load_section, net),
        # executor.submit(load_static_gen_section, net),
    ]

    for future in futures:
        future.result()

# 环节6：加载开关
load_switch_section(net)

# 环节7: 设置平衡节点
pp.create_ext_grid(net, bus=1)

# 环节8：检查网络
check_network_data(net)


Starting Bus Section
Bus measurements loaded.
Bus Section completed

Starting Transformer Section
Starting Line Section
Starting Generator Section
Generator measurements loaded.
Generator Section completed

Starting Load Section
Line measurements loaded.
Line Section completed

Load measurements loaded.
Load Section completed

Transformer measurements loaded.
Transformer Section completed

Starting Switch Section
Switch measurements loaded.
Switch Section completed

Starting network data check
network data check completed and power flow passed


In [6]:
from copy import deepcopy
# 事故前潮流已计算，现假设昇光变电所.1号主变跳闸，计算事故后潮流
test_net = deepcopy(net)
test_net.trafo3w.loc[net.trafo3w['name'] == "昇光变电所.1号主变", 'in_service'] = False
base_power = test_net.res_trafo3w.loc[net.trafo3w['name'] == "昇光变电所.1号主变", 'p_hv_mw'][0]
pp.runpp(test_net) #事故后潮流
DET_P = 0.2
det_trafo3w = ((test_net.res_trafo3w["p_hv_mw"] - net.res_trafo3w["p_hv_mw"])/base_power).abs()
det_line = ((test_net.res_line["p_from_mw"] - net.res_line["p_from_mw"])/base_power).abs()
# 筛选潮流变化大的
nminus1_cases = {
    "trafo3w": {"index": det_trafo3w[det_trafo3w > 0.2].index.values},
    "line":{"index": det_line[det_line > 0.2].index.values}
    }  

element_limits = pp.contingency.get_element_limits(test_net)  

# 2. 运行应急分析  

pp.contingency.run_contingency(test_net, nminus1_cases)  
  
# 3. 报告违反限制的结果  

{'bus': {'index': array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
         17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
         34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46]),
  'max_vm_pu': array([1.        , 1.        , 0.96139853, 0.95556052, 0.95556052,
         0.96139853, 0.96139853, 0.96139853, 1.        , 1.        ,
         0.96139853, 0.96139853, 0.95556052, 0.96139853, 0.95556052,
         0.96139853, 0.96139853, 0.96139853, 0.96139853, 0.96139853,
         0.96139853, 1.        , 1.        , 0.96139853, 0.95556052,
         0.95556052, 0.95556052, 0.95556052, 0.96139853, 0.96139853,
         0.96139853, 0.96139853, 1.        , 0.96139853,        nan,
         1.        , 0.96139853, 0.97265843, 1.        , 0.96139853,
         0.97197791, 0.95557093, 0.95685275, 0.95556052, 0.94123463,
         0.95324258, 0.95556052]),
  'min_vm_pu': array([1.        , 1.        , 0.96104067, 0.95519869, 0.95519869,
         0.96104

In [7]:
net.res_trafo3w

,p_hv_mw,q_hv_mvar,p_mv_mw,q_mv_mvar,p_lv_mw,q_lv_mvar,pl_mw,ql_mvar,i_hv_ka,i_mv_ka,i_lv_ka,vm_hv_pu,va_hv_degree,vm_mv_pu,va_mv_degree,vm_lv_pu,va_lv_degree,va_internal_degree,vm_internal_pu,loading_percent
0,560.634562,103.313675,-559.898165,-69.582515,0.0000,0.0000,0.736397,33.731160,0.658265,1.495590,0.000000,1.00000,0.000000,0.990014,-3.287175,0.994589,-1.635379,-1.635379,0.994589,47.506200
1,560.018794,129.984479,-558.992710,-43.186287,-0.2853,-52.8500,0.740784,33.948191,0.663844,1.486188,0.880155,1.00000,0.000000,0.990014,-3.287175,0.990521,-1.631841,-1.634203,0.993233,47.908841
2,559.959087,133.185974,-558.869978,-40.018558,-0.3472,-59.1640,0.741908,34.003416,0.664623,1.485240,0.985796,1.00000,0.000000,0.990014,-3.287175,0.990032,-1.631540,-1.634103,0.993070,47.965025
3,130.296498,-0.826795,-130.056000,9.850800,0.0000,0.0000,0.240498,9.024005,0.347337,0.694381,0.000000,0.98448,-4.585122,0.985874,-8.496635,0.984595,-6.542480,-6.542480,0.984595,55.147193
4,123.267685,21.431187,-122.795900,18.202600,-0.2414,-31.0874,0.230385,8.546387,0.333523,0.663251,0.528311,0.98448,-4.585122,0.982365,-8.306172,0.970686,-6.434802,-6.440867,0.978826,52.953859
